In [4]:
!pip install nltk
!pip install mechanize
!pip install html2text

In [6]:
import re
import ssl
import mechanize
import pandas as pd
from requests import get
from bs4 import BeautifulSoup
from html2text import html2text

titles = [] #título da matéria
titlesFolha = []
links = [] #link da matéria
linksFolha = []

linksTexto = [] #todos os links encontrados nas páginas
linksTextos = []
fullPage = [] # todo o texto das páginas
fullPages = []

def clean_html(html):
    cleaned = re.sub(r"(?is)<(script|style).*?>.*?(</\1>)", "", html.strip())
    cleaned = re.sub(r"(?s)<!--(.*?)-->[\n]?", "", cleaned)
    cleaned = re.sub(r"(?s)<.*?>", " ", cleaned)
    cleaned = re.sub(r"&nbsp;", " ", cleaned)
    cleaned = re.sub(r"  ", " ", cleaned)
    cleaned = re.sub(r"  ", " ", cleaned)
    return cleaned.strip()

def beaultifulSoup(url):
    ssl._create_default_https_context = ssl._create_unverified_context
    response = get(url)
    html = BeautifulSoup(response.text, 'html.parser')
    return html

def getMenuUol(url):
    html = beaultifulSoup(url)
    allNews = html.find_all('li', class_ = 'menu-item')
    for news in allNews:
        title = news.a.text
        titles.append(title)

        link = news.a['href']
        links.append(link)

def getMenuFolha(url):
    html = beaultifulSoup(url)
    allNews = html.find_all('li', class_ = 'c-site-nav__item')
    for news in allNews:
        title = news.a.text
        titlesFolha.append(title)

        link = re.sub(r"^\//|\$", "https://", str(news.a['href'].strip()))
        linksFolha.append(link)


def getPegaText(links):
    try:
        for link in links:
            br = mechanize.Browser()
            br.set_handle_robots(False)
            br.addheaders = [('User-agent', 'Firefox')]
            html = br.open(link).read().decode('utf-8')
            cleanhtml = clean_html(html)
            fullPages.append(html2text(cleanhtml))
            html_soup = beaultifulSoup(link)
            for link in html_soup.find_all('a'):
                linksTextos.append(link.get('href'))
    except:
        print('houve um erro')

def getPegaTextSinleLink(link):
    br = mechanize.Browser()
    br.set_handle_robots(False)
    br.addheaders = [('User-agent', 'Firefox')]
    html = br.open(link).read().decode('utf-8')
    cleanhtml = clean_html(html)
    fullPage.append(html2text(cleanhtml))
    html_soup = beaultifulSoup(link)
    for link in html_soup.find_all('a'):
        linksTexto.append(link.get('href'))

getMenuUol('https://www.uol.com.br/vivabem/')
test_df = pd.DataFrame({'title': titles,'link': links})
print(test_df)

getPegaTextSinleLink(links[28])
print(len(linksTexto))

getPegaText(links)
test_df = pd.DataFrame({'text': fullPages})
print(test_df)
test_df.to_csv('scrapy.csv')

                 title                                               link
0           Home UOL                              https://www.uol.com.br/
1           Produtos    https://clicklogger.rm.uol.com.br/?prd=11&grp=...
2           Notícias                         https://noticias.uol.com.br/
3             Carros                       https://www.uol.com.br/carros/
4           Economia                         https://economia.uol.com.br/
5              Folha                        https://www.folha.uol.com.br/
6            Esporte                      https://www.uol.com.br/esporte/
7             Splash                       https://www.uol.com.br/splash/
8       TV e Famosos                       https://tvefamosos.uol.com.br/
9           Universa                     https://www.uol.com.br/universa/
10           VivaBem                      https://www.uol.com.br/vivabem/
11              Tilt                         https://www.uol.com.br/tilt/
12              Ecoa                  